In [2]:
sc.install_pypi_package('pyarrow==0.10.0')
sc.install_pypi_package('matplotlib')
sc.install_pypi_package('geopandas')
sc.install_pypi_package('descartes')
sc.install_pypi_package('shapely')
sc.install_pypi_package('sklearn')
sc.install_pypi_package('requests')
sc.install_pypi_package('hashids')
sc.install_pypi_package('pandas==1.0.0')
sc.install_pypi_package('scipy')
sc.install_pypi_package('numpy')
sc.install_pypi_package('geopy')
sc.install_pypi_package('pytz')
sc.install_pypi_package('s3fs==2021.07.0')
sc.install_pypi_package('boto3')
sc.install_pypi_package('pygeohash')
sc.install_pypi_package('aiobotocore==1.3.3')
sc.install_pypi_package('scikit-learn')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pyarrow-0.10.0-cp36-cp36m-manylinux1_x86_64.whl (11.6 MB)

  Using cached matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl (11.5 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached Pillow-8.4.0-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached kiwisolver-1.3.1-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached pyparsing-3.1.0-py3-none-any.whl (102 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.5
    Not uninstalling numpy at /usr/local/lib64/python3.6/site-packages, outside environment /tmp/1688663670645-0
    Can't uninstall 'numpy'. No files were found to uninstall.

  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)
  Using cached Shapely-1.8.5.post1-cp36-cp36m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.0 MB)
  Using cache

In [4]:
import sys
import os
import pytz
import math
from functools import reduce
from dateutil import tz
import datetime
from hashids import Hashids
# from s3fs.core import S3FileSystem
import shapely.geometry as sg
from shapely.geometry import Point, Polygon, shape
from shapely import wkt
import shapely.speedups
from geopy.distance import geodesic
import geopandas as gpd
import pandas as pd
import numpy as np
import time
import cgitb
from itertools import islice
from pyspark import Row, SQLContext
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, DecimalType, StructType, StructField
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import row_number, rank, split, mean, hour, pandas_udf, PandasUDFType, weekofyear, udf, struct,lag, col, when, lit, first, sha1, concat, lpad, substring, regexp_replace, countDistinct,to_date,month, date_format, isnull, isnan
import pyspark.sql.functions as SparkFunc
from pyspark.sql.window import Window
from pyspark import SparkContext, SparkConf
from scipy.spatial import cKDTree as cKDTree
from sklearn.neighbors import KDTree, BallTree
from s3fs.core import S3FileSystem
import boto3
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import PipelineModel
from pyspark.sql import DataFrame
import pygeohash as pgh

sql_context = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## weekly gas price

In [ ]:
gas_stations = spark.read.format("csv").option("header", "True").option("delimiter", ",").load(
    '/gas_price_202101_202302.csv').filter(col("Latitude").isNotNull()).withColumn(
    "Date",date_format(to_date(col("Date"),"yyyy-MM-dd"),"MM/dd/yyyy")).withColumn(
    'Month', split(col('Date'), '/').getItem(0)).withColumn(
    'Year', split(col('Date'), '/').getItem(2)).withColumn(
    "Week", date_format(to_date("Date", "MM/dd/yyyy"), "w")).withColumn(
    'Price',col('Price').cast(FloatType()))
gas_stations

In [27]:
gas_stations.withColumn("Date",date_format(to_date(col("Date"),"MM/dd/yyyy"),"yyyy-MM-dd")).groupBy('Date').agg(
    SparkFunc.avg('Price').alias('avg_gas_price')).orderBy('Date').toPandas().to_csv(
    '/weekly_avg_gas_price.csv', index=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## KD tree to identify gas stations within a buffer

In [32]:
from datetime import date

# week of year
def week1_start_ordinal(year):
    jan1 = date(year, 1, 1)
    jan1_ordinal = jan1.toordinal()
    jan1_weekday = jan1.weekday()
    week1_start_ordinal = jan1_ordinal - ((jan1_weekday + 1) % 7)
    return week1_start_ordinal
# only consider 2022
def week_from_date(Year, Month, Day):
    date_object = date(Year, Month, Day)
    date_ordinal = date_object.toordinal()
    year = date_object.year
    week = ((date_ordinal - week1_start_ordinal(year)) // 7) + 1
    if week > 52:
        if year == 2021:
            #year += 1
            week = 1
    return week

week_from_date_udf = udf(week_from_date, IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
### decode geohash to lat and lon
def geohash_decode_lat(geohash7):
    if geohash7 is not None:
        return pgh.decode(geohash7)[0]
    
geohash_decode_lat_udf = udf(geohash_decode_lat, FloatType())

def geohash_decode_long(geohash7):
    if geohash7 is not None:
        return pgh.decode(geohash7)[1]
    
geohash_decode_long_udf = udf(geohash_decode_long, FloatType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#transformation of geo coordinate system
@pandas_udf('float',PandasUDFType.SCALAR)
def lng_to_meters(longitude):
    if longitude is not None:
        origin_shift = np.pi * 6378137
        easting = longitude * origin_shift / 180.0
        return easting
@pandas_udf('float',PandasUDFType.SCALAR)
def lat_to_meters(latitude):
    if latitude is not None:
        origin_shift = np.pi * 6378137
        northing = np.log(np.tan((90 + latitude) * np.pi / 360.0)) * origin_shift / np.pi
        return  northing

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
gas_stations = spark.read.format("csv").option("header", "True").option("delimiter", ",").load(
    '/gas_price_202101_202302.csv').filter(col("Latitude").isNotNull()).withColumn(
    "Date",date_format(to_date(col("Date"),"yyyy-MM-dd"),"MM/dd/yyyy")).withColumn(
    'Month', split(col('Date'), '/').getItem(0).cast(IntegerType())).withColumn(
    'Day', split(col('Date'), '/').getItem(1).cast(IntegerType())).withColumn(
    'Year', split(col('Date'), '/').getItem(2).cast(IntegerType())).withColumn(
    "Week", week_from_date_udf(col("Year"), col("Month"), col("Day"))).withColumn(
    'Price',col('Price').cast(FloatType())).filter((col("Year") == 2022) | ((col("Year") == 2021) & (col("Week") == 1)))
gas_stations

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Date: string, Station ID: string, Latitude: string, Longitude: string, Price: float, Gas Brand: string, Month: int, Day: int, Year: int, Week: int]

In [34]:
gas_stations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+-----------+------------+-----+---------+-----+---+----+----+
|      Date|Station ID|   Latitude|   Longitude|Price|Gas Brand|Month|Day|Year|Week|
+----------+----------+-----------+------------+-----+---------+-----+---+----+----+
|12/26/2021|    623483|38.91129558|-77.01066949|3.372|    Exxon|   12| 26|2021|   1|
|12/26/2021|    292229|38.91407013|-77.01608276|3.386|    Exxon|   12| 26|2021|   1|
|12/26/2021|    489982|38.91365882|-77.01731475|3.316| Marathon|   12| 26|2021|   1|
|12/26/2021|    623380|38.91337917|-77.01813239|3.375|   Valero|   12| 26|2021|   1|
|12/26/2021|    444106|38.90776661| -77.0236775|3.599|   Sunoco|   12| 26|2021|   1|
|12/26/2021|    489980|38.92666018|-77.02622062|3.599|   Sunoco|   12| 26|2021|   1|
|12/26/2021|    298560|38.90111558|-76.98307067|3.315|    Mobil|   12| 26|2021|   1|
|12/26/2021|    560313|38.89540373| -77.0032514|4.299|    Exxon|   12| 26|2021|   1|
|12/26/2021|    608841|38.91672501|-76.98117103|3.296|    Exxon| 

In [39]:
spark.conf.set("spark.sql.broadcastTimeout" ,"-1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
year = 2022

for month in range(1,13):
    if month == 2:
        num_days = 28
    elif month in [4,6,9,11]:
        num_days = 30
    else:
        num_days = 31
    #device list with geohash of home and work location
    WMA_device_list_month = spark.read.format("csv").option("header", "True").option("delimiter", ",").load(
        '/device_list/WMA_device_list_%s%02d.csv' % (year, month))
    #decode to lat and lon
    WMA_device_list_month = WMA_device_list_month.withColumn(
        'home_lat', geohash_decode_lat_udf(col('h_geohash'))).withColumn(
        'home_long', geohash_decode_long_udf(col('h_geohash'))).withColumn(
        'work_lat', geohash_decode_lat_udf(col('w_geohash'))).withColumn(
        'work_long', geohash_decode_long_udf(col('w_geohash')))
    #transfer to x and y
    WMA_device_list_month = WMA_device_list_month.withColumn(
        'X_h', lng_to_meters(WMA_device_list_month.home_long)).withColumn(
        'Y_h', lat_to_meters(WMA_device_list_month.home_lat)).withColumn(
        'X_w', lng_to_meters(WMA_device_list_month.work_long)).withColumn(
        'Y_w', lat_to_meters(WMA_device_list_month.work_lat))
    
    for weekofyear in range(week_from_date(year, month, 1), week_from_date(year, month, num_days)+1):
        #gas_stations filter & integration
        gas_stations_weekly = gas_stations.filter(col('Week')== weekofyear)
        gas_stations_weekly = gas_stations_weekly.withColumn(
            'gas_station_x', lng_to_meters(gas_stations_weekly.Longitude.cast(FloatType()))).withColumn(
            'gas_station_y', lat_to_meters(gas_stations_weekly.Latitude.cast(FloatType())))
        gas_stations_weekly = gas_stations_weekly.toPandas()
        
        #kd tree
        gas_stations_weekly_kdtree = KDTree(gas_stations_weekly[['gas_station_x','gas_station_y']])
        gas_stations_weekly_kdtree_bc = sc.broadcast(gas_stations_weekly_kdtree)
        gas_stations_weekly_bc = sc.broadcast(gas_stations_weekly)
        
        WMA_device_gas_weekly = WMA_device_list_month.select('device_id')

        for dis in range(2,6):

            # Function to find the gas stations within a certain distance
            @pandas_udf(
                'device_id string, X_h float, Y_h float, X_w float, Y_w float, h_gs_count_%smi integer, \
                h_gs_avg_price_%smi float, w_gs_count_%smi integer, w_gs_avg_price_%smi float' % (dis, dis, dis, dis), 
                PandasUDFType.GROUPED_MAP)
            def gas_station_identify(pdf):
                gas_station_ind_h = gas_stations_weekly_kdtree_bc.value.query_radius(
                    np.array(pdf.loc[:,['X_h','Y_h']]), r=dis*1609.344)
                #number of gasoline stations
                pdf['h_gs_count_%smi' % dis] = len(gas_station_ind_h[0])
                #average gasoline price
                if len(gas_station_ind_h) != 0:
                    pdf['h_gs_avg_price_%smi' % dis] = gas_stations_weekly_bc.value.loc[gas_station_ind_h[0].reshape(
                        1, gas_station_ind_h[0].shape[0]).tolist()[0],'Price'].reset_index(drop=True).mean()
    
                if math.isnan(pdf.loc[:,'X_w']) == False:
                    gas_station_ind_w = gas_stations_weekly_kdtree_bc.value.query_radius(
                        np.array(pdf.loc[:,['X_w','Y_w']]), r=dis*1609.344)
                    pdf['w_gs_count_%smi' % dis] = len(gas_station_ind_w[0])
                    
                    if len(gas_station_ind_w) != 0:
                        pdf['w_gs_avg_price_%smi' % dis] = gas_stations_weekly_bc.value.loc[gas_station_ind_w[0].reshape(
                            1, gas_station_ind_w[0].shape[0]).tolist()[0],'Price'].reset_index(drop=True).mean()
                else:
                    pdf['w_gs_count_%smi' % dis] = float("nan")
                    pdf['w_gs_avg_price_%smi' % dis] = float("nan")        
    
                return pdf
    
            WMA_device_gas_weekly_bydis = WMA_device_list_month.select(
                'device_id','X_h','Y_h','X_w','Y_w').groupby('device_id').apply(gas_station_identify) 
    

            WMA_device_gas_weekly = WMA_device_gas_weekly.join(WMA_device_gas_weekly_bydis.drop('X_h','Y_h','X_w','Y_w'), 
                                                               on = 'device_id', how = 'inner')
    
        WMA_device_gas_weekly.toPandas().to_csv(
            '/WMA_device_gas_week%02d_%s%02d.csv' % (
                weekofyear, year, month), index=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…